In [ ]:
from research.weight_estimation.akpd_utils.akpd import AKPD
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
import json
import os
import  pandas as pd
import numpy as np
import matplotlib.patches as patches
from tqdm import tqdm
from utils import utils, data_prep
import cv2
from PIL import Image

import matplotlib.pyplot as plt


import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

In [ ]:
import multiprocessing as mp



In [ ]:
mp.cpu_count()

In [ ]:
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))
#credentials = json.load(open("/root/jane/cv_research/jane/deploy/data_warehouse_sql_credentials.json"))



In [ ]:
def get_annotation_data(pen_id, start_date, end_date):
    sql = """
            SELECT 
                group_id,
                url_key,
                camera_metadata,
                CASE 
                    WHEN url_key like '%%left%%' THEN left_crop_metadata
                    ELSE right_crop_metadata          
                END AS crop_metadata,
                annotation,
                captured_at

            FROM 
                prod.crop_annotation 
            WHERE 
                (captured_at BETWEEN '{}' AND '{}') AND 
                (group_id in {}) AND
                (annotation_state_id IN (7)) AND
                (service_id = 1);
            """.format(start_date, end_date, pen_id)
    return rds_access_utils.extract_from_database(sql)



In [ ]:

annotation_data = get_annotation_data("('56', '60', '37', '66', '85', '86')", 
                                      "2020-04-01", 
                                      "2020-05-01")

In [ ]:
annotation_data.shape

In [ ]:
aws_credentials = json.load(open(os.environ['AWS_CREDENTIALS']))

akpd = AKPD(aws_credentials)

In [ ]:
# annotation
# create AKPD SageMaker client
aws_credentials = json.load(open(os.environ['AWS_CREDENTIALS']))

akpd = AKPD(aws_credentials)

In [ ]:
def add_keypoints_feature(df): 
    df['kps'] = df.apply(lambda row: akpd.predict_keypoints(row.url_key, row.url_key,
                                                            row.crop_metadata, row.crop_metadata, 
                                                            row.camera_metadata), axis=1)
    return df

In [ ]:
def parallelize_dataframe(df, func, n_cores=12):
    df_split = np.array_split(df, n_cores)
    pool = mp.Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df



In [ ]:
%%time
annotation_data_akpd = parallelize_dataframe(annotation_data, add_keypoints_feature, n_cores=12)




In [ ]:
annotation_data_akpd.shape

In [ ]:
# only can run once. 
# clean kps column since left and right crop is duplicated
# annotation_data_akpd['kps'] = annotation_data_akpd['kps'].apply(lambda x: x[0]["leftCrop"])

In [ ]:
annotation_data_akpd.head(4)

In [ ]:
annotation_data_akpd.to_pickle("annotation_data_akpd_2020_05_27.pkl")